In [16]:
import os
import glob

def rename_images_to_sequential(folder_path, prefix='image', ext='jpg'):
    # 모든 jpg 파일 경로 가져오기
    image_paths = sorted(glob.glob(os.path.join(folder_path, f'*.{ext}')))

    # 리네이밍
    for idx, old_path in enumerate(image_paths):
        new_filename = f"{prefix}_{idx+1:04d}.{ext}"
        new_path = os.path.join(folder_path, new_filename)
        os.rename(old_path, new_path)
        # print(f"{os.path.basename(old_path)} → {new_filename}")

# 실행
rename_images_to_sequential("/home/najo/NAS/Computer_Vision/image")

In [22]:
import cv2
import numpy as np
import os
import glob
import shutil
import open3d as o3d

# =========================
# 조절 가능한 파라미터 (Adjustable Parameters)
# =========================
image_folder = '/home/najo/NAS/Computer_Vision/image'  # 이미지 폴더 경로 (필요시 변경)
output_folder = './sfm_output'                         # 결과 저장 폴더
SUPPORTED_EXT = ('.jpg', '.JPG')
LOWE_RATIO_THRESH = 0.95                            # Lowe의 ratio test 임계값
MIN_MATCH_COUNT = 5                                       # 초기 매칭 최소 개수
MIN_PNP_POINTS = 4                                   # PnP 최소 대응점 수 (최소 4개)
RANSAC_THRESHOLD = 3.0                               # Essential Matrix 추정 RANSAC 임계값

# 추가 최적화 파라미터: Outlier Removal (3D 포인트 클라우드 필터링)
OUTLIER_NB_NEIGHBORS = 30
OUTLIER_STD_RATIO = 3.0

# =========================
# 출력 폴더 생성
# =========================
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# =========================
# 이미지 로딩 및 K 행렬 추정 (첫 이미지 기준)
# =========================
img_files = sorted(glob.glob(os.path.join(image_folder, '*')))[::-1]
img_files = [f for f in img_files if f.lower().endswith(SUPPORTED_EXT)]
if len(img_files) < 2:
    print("Error: Need at least two images for SfM.")
    exit()

images = []
for f in img_files:
    img = cv2.imread(f)
    if img is not None:
        images.append(img)
print(f"Loaded {len(images)} images.")

K_path = os.path.join(image_folder, 'K.txt')
K = np.loadtxt(K_path)

# =========================
# SIFT를 통한 특징점 추출
# =========================
sift = cv2.SIFT_create()
keypoints_list = []
descriptors_list = []
print("Extracting SIFT features...")
for i, img in enumerate(images):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray, None)
    keypoints_list.append(kp)
    descriptors_list.append(des)
    print(f"Image {i}: {len(kp)} features")

# =========================
# FLANN 기반 매칭 (Lowe's ratio test 적용)
# =========================
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
all_matches = {}
num_images = len(images)
print("Matching features between image pairs...")
for i in range(num_images):
    for j in range(i + 1, num_images):
        if descriptors_list[i] is not None and descriptors_list[j] is not None:
            matches = flann.knnMatch(descriptors_list[i], descriptors_list[j], k=2)
            good_matches = []
            for m, n in matches:
                if m.distance < LOWE_RATIO_THRESH * n.distance:
                    good_matches.append(m)
            if len(good_matches) > MIN_MATCH_COUNT:
                all_matches[(i, j)] = good_matches
                print(f"Matches between image {i} and {j}: {len(good_matches)}")

# =========================
# 초기 이미지 쌍 선택 및 초기화 (Step I & II)
# =========================
best_pair = None
max_matches = 0
for pair, matches in all_matches.items():
    if len(matches) > max_matches:
        max_matches = len(matches)
        best_pair = pair
if best_pair is None:
    print("Error: No suitable initial pair found.")
    exit()
i1, i2 = best_pair
initial_matches = all_matches[best_pair]
print(f"Selected initial pair: Image {i1} and {i2} with {len(initial_matches)} matches")

pts1 = np.float32([keypoints_list[i1][m.queryIdx].pt for m in initial_matches]).reshape(-1, 1, 2)
pts2 = np.float32([keypoints_list[i2][m.trainIdx].pt for m in initial_matches]).reshape(-1, 1, 2)

# Essential Matrix 추정 (RANSAC 적용)
E, mask_e = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.999, threshold=RANSAC_THRESHOLD)
if E is None:
    print("Error: Could not estimate Essential Matrix.")
    exit()
inlier_matches = [m for i, m in enumerate(initial_matches) if mask_e[i]]
pts1_inlier = np.float32([keypoints_list[i1][m.queryIdx].pt for m in inlier_matches]).reshape(-1, 2)
pts2_inlier = np.float32([keypoints_list[i2][m.trainIdx].pt for m in inlier_matches]).reshape(-1, 2)
print(f"Essential Matrix inliers: {len(pts1_inlier)}")

# 포즈 복원 (recoverPose)
points, R, t, mask_rp = cv2.recoverPose(E, pts1_inlier, pts2_inlier, K)
print(f"Recovered pose: {points} points in front of cameras")

# 삼각측량을 통한 초기 3D 포인트 복원 (Step IV)
P1 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
P2 = K @ np.hstack((R, t))
points4D = cv2.triangulatePoints(P1, P2, pts1_inlier.T, pts2_inlier.T)
points3D = (points4D / points4D[3])[:3].T
print(f"Triangulated {len(points3D)} 3D points.")

# 데이터 구조 초기화 (색상 정보 포함)
camera_poses = {}
camera_poses[i1] = (np.eye(3), np.zeros((3, 1)))
camera_poses[i2] = (R, t)
registered_images = {i1, i2}
point_cloud = {}
point_visibility = {}
keypoint_to_3d = {}
point_colors = {}  # 각 3D 포인트의 색상 (RGB, 0~1)
for idx, m in enumerate(inlier_matches):
    pt_idx = len(point_cloud)
    point_cloud[pt_idx] = points3D[idx]
    kp_idx1 = m.queryIdx
    kp_idx2 = m.trainIdx
    point_visibility[pt_idx] = [(i1, kp_idx1), (i2, kp_idx2)]
    keypoint_to_3d[(i1, kp_idx1)] = pt_idx
    keypoint_to_3d[(i2, kp_idx2)] = pt_idx
    # 색상 정보: 초기 이미지(i1)에서 keypoint 위치의 색상 (BGR -> RGB, 정규화)
    x, y = keypoints_list[i1][kp_idx1].pt
    color_bgr = images[i1][int(round(y)), int(round(x))]
    color_rgb = color_bgr[::-1] / 255.0
    point_colors[pt_idx] = color_rgb

# 중간 결과 저장 함수 (각 뷰별 ply 파일 저장)
def save_intermediate_results(view_count):
    pcd = o3d.geometry.PointCloud()
    pts = np.array([pt for pt in point_cloud.values()])
    pcd.points = o3d.utility.Vector3dVector(pts)
    colors = np.array([point_colors[pt] for pt in sorted(point_cloud.keys())])
    pcd.colors = o3d.utility.Vector3dVector(colors)
    ply_filename = os.path.join(output_folder, f'point_cloud_{view_count}view.ply')
    o3d.io.write_point_cloud(ply_filename, pcd)
    print(f"Intermediate point cloud saved to {ply_filename}")

# =========================
# 초기 2-view 결과 저장 (Step I-II 완료)
# =========================
save_intermediate_results(len(registered_images))
# =========================
# Growing Step: 남은 이미지들에 대해 PnP를 통한 카메라 포즈 추정 (Step V)
# =========================
remaining_images = set(range(num_images)) - registered_images
while remaining_images:
    best_next_img = -1
    best_match_count = 0
    best_obj_pts = []
    best_img_pts = []
    for next_idx in remaining_images:
        temp_obj_pts = []
        temp_img_pts = []
        for reg_idx in registered_images:
            pair = tuple(sorted((next_idx, reg_idx)))
            if pair in all_matches:
                for m in all_matches[pair]:
                    if pair[0] == next_idx:
                        kp_idx_next = m.queryIdx
                        kp_idx_reg = m.trainIdx
                    else:
                        kp_idx_next = m.trainIdx
                        kp_idx_reg = m.queryIdx
                    if (reg_idx, kp_idx_reg) in keypoint_to_3d:
                        pt_idx = keypoint_to_3d[(reg_idx, kp_idx_reg)]
                        if pt_idx in point_cloud:
                            temp_obj_pts.append(point_cloud[pt_idx])
                            temp_img_pts.append(keypoints_list[next_idx][kp_idx_next].pt)
        if len(temp_obj_pts) > best_match_count and len(temp_obj_pts) >= MIN_PNP_POINTS:
            best_match_count = len(temp_obj_pts)
            best_next_img = next_idx
            best_obj_pts = np.array(temp_obj_pts, dtype=np.float32)
            best_img_pts = np.array(temp_img_pts, dtype=np.float32)
    if best_next_img == -1:
        print("No more images can be registered. Stopping.")
        break
    print(f"Adding Image {best_next_img} with {best_match_count} correspondences.")
    if len(best_obj_pts) < 4:
        print(f"Not enough points for PnP for Image {best_next_img}, skipping.")
        remaining_images.remove(best_next_img)
        continue
    ret, rvec, tvec, inliers = cv2.solvePnPRansac(
        best_obj_pts, best_img_pts, K, None,
        iterationsCount=500, reprojectionError=10.0, confidence=0.99
    )
    if ret:
        R_new, _ = cv2.Rodrigues(rvec)
        camera_poses[best_next_img] = (R_new, tvec)
        registered_images.add(best_next_img)
        remaining_images.remove(best_next_img)
        print(f"Image {best_next_img} registered successfully.")
        save_intermediate_results(len(registered_images))
    else:
        print(f"PnP failed for Image {best_next_img}. Removing from candidates.")
        remaining_images.remove(best_next_img)

# 최종 결과 저장 (현재까지의 3D 포인트 클라우드)
final_ply_filename = os.path.join(output_folder, 'point_cloud_final.ply')
pcd = o3d.geometry.PointCloud()
final_pts = np.array([pt for pt in point_cloud.values()])
pcd.points = o3d.utility.Vector3dVector(final_pts)
final_colors = np.array([point_colors[pt] for pt in sorted(point_cloud.keys())])
pcd.colors = o3d.utility.Vector3dVector(final_colors)
pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=OUTLIER_NB_NEIGHBORS, std_ratio=OUTLIER_STD_RATIO)
o3d.io.write_point_cloud(final_ply_filename, pcd)
print(f"Final colored point cloud saved to {final_ply_filename}")

# 모든 이미지에 대해 ply 파일 생성 (등록되지 않은 이미지도 포함)
for i in range(num_images):
    ply_filename = os.path.join(output_folder, f'point_cloud_{i+1}view.ply')
    if i in registered_images:
        # 등록된 이미지: 이미 중간 결과로 저장되어 있을 수 있음. 여기서는 덮어쓰기.
        shutil.copy(final_ply_filename, ply_filename)
        print(f"Saved ply for registered image {i} as {ply_filename}")
    else:
        # 미등록 이미지는 최종 ply 파일을 복사하여 사용
        new_filename = os.path.join(output_folder, f'point_cloud_{i+1}view_failed.ply')
        shutil.copy(final_ply_filename, new_filename)
        print(f"Saved ply for unregistered image {i} as {new_filename}")

print("\nSfM process finished (without bundle adjustment, full ply saving for all images).")


Loaded 32 images.
Extracting SIFT features...
Image 0: 320 features
Image 1: 1317 features
Image 2: 1226 features
Image 3: 832 features
Image 4: 779 features
Image 5: 843 features
Image 6: 382 features
Image 7: 328 features
Image 8: 399 features
Image 9: 1317 features
Image 10: 345 features
Image 11: 428 features
Image 12: 348 features
Image 13: 239 features
Image 14: 502 features
Image 15: 353 features
Image 16: 298 features
Image 17: 340 features
Image 18: 232 features
Image 19: 277 features
Image 20: 1067 features
Image 21: 605 features
Image 22: 585 features
Image 23: 267 features
Image 24: 440 features
Image 25: 665 features
Image 26: 622 features
Image 27: 376 features
Image 28: 651 features
Image 29: 409 features
Image 30: 353 features
Image 31: 342 features
Matching features between image pairs...
Matches between image 0 and 1: 155
Matches between image 0 and 2: 141
Matches between image 0 and 3: 164
Matches between image 0 and 4: 163
Matches between image 0 and 5: 167
Matches 

In [ ]:
import open3d as o3d

# PLY 파일 경로
ply_file_path = "/home/najo/NAS/Computer_Vision/data/output_ply_by_process/step_0031_map.ply"

# PLY 파일 로드
pcd = o3d.io.read_point_cloud(ply_file_path)
print(f"불러온 포인트 클라우드의 포인트 수: {len(pcd.points)}")

# 포인트 클라우드 시각화 (창 내 'q' 키로 종료)
o3d.visualization.draw_geometries([pcd], window_name="PLY 파일 시각화")


불러온 포인트 클라우드의 포인트 수: 43
[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
